In [3]:
!pip show pyts


In [4]:
!pip install numpy pandas matplotlib pyts scikit-learn

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------ --------------- 1.6/2.5 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --------------------------------- ------ 2.4/2.8 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 10.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   --- ------------------------------------ 2.4/30.3 MB 12.2 MB/s eta 0:00:03
   ----- ---------------------------------- 4.5/30.3 MB 11.2 MB/s eta 0:00:03
   -------- ------------------------------- 6.8/30.3 MB 11.3 MB/s eta 0:00:03
   ------------ --------------------------- 9.2/30.3 MB 11.4 MB/s eta 0:00:02
   --------------- ------------------------ 11.8/30.3 MB 11.5 MB/s eta 0:00:02
   ------------------ --------------------- 14.2/30.3 MB 11.5 MB/s eta 0:00:02
   ------------

In [1]:
!git clone https://github.com/eliotwalt/gaf-cnn.git

Cloning into 'gaf-cnn'...


In [2]:
%cd gaf-cnn/

f:\Time_series\gaf-cnn


f:\miniconda\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pip install torch torchsummary numpy matplotlib

^C


  Using cached torchsummary-1.5.1-py3-none-any.whl.metadata (296 bytes)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.2 MB 1.4 MB/s eta 0:02:30
   ---------------------------------------- 0.1/204.2 MB 1.2 MB/s eta 0:02:53
   ---------------------------------------- 0.2/204.2 MB 1.8 MB/s eta 0:01:56
   ---------------------------------------- 0.3/204.2 MB 1.8 MB/s eta 0:01:56
   ---------------------------------------- 0.4/204.2 MB 1.9 MB/s eta 0:01:46
   ---------------------------------------- 0.5/204.2 MB 2.0 MB/s eta 0:01:44
   ---------------------------------------- 0.5/204.2 MB 2.0 MB/s eta 0:01:44
   ---------------------------------------- 0.6/204.2 M


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pwd
%cd ..

f:\Time_series


f:\miniconda\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
!pip show pyts

Name: pyts
Version: 0.13.0
Summary: A python package for time series classification
Home-page: https://github.com/johannfaouzi/pyts
Author: 
Author-email: 
License: new BSD
Location: c:\users\thongtrươngxuan\appdata\local\programs\python\python310\lib\site-packages
Requires: joblib, numba, numpy, scikit-learn, scipy
Required-by: 


In [ ]:
import pandas as pd
import numpy as np
import os
from pyts.image import GramianAngularField
from sklearn.preprocessing import MinMaxScaler
from PIL import Image

seq_len = 96
input_file = 'dataset/sp500/GOOG_filtered.csv' 
output_dir = 'gasf_png_rgb'
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(input_file, usecols=['date', 'Close'])
df['Date'] = pd.to_datetime(df['Date'])

for symbol in df['Symbol'].unique():
    df_symbol = df[df['Symbol'] == symbol].sort_values('Date')
    close_prices = df_symbol['Close'].values

    # scale
    scaler = MinMaxScaler(feature_range=(-1, 1))
    normalized = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

    # GASF
    gasf = GramianAngularField(image_size=seq_len, method='summation')
    symbol_dir = os.path.join(output_dir, symbol)
    os.makedirs(symbol_dir, exist_ok=True)

    for i in range(len(normalized) - seq_len + 1):
        window = normalized[i:i + seq_len].reshape(1, -1)
        gasf_image = gasf.fit_transform(window)[0]

        gasf_scaled = ((gasf_image - np.min(gasf_image)) / (np.max(gasf_image) - np.min(gasf_image)) * 255).astype(np.uint8)

        
        rgb_image = np.stack([gasf_scaled]*3, axis=-1)

        img = Image.fromarray(rgb_image, mode='RGB')
        date_str = df_symbol.iloc[i + seq_len - 1]['Date'].strftime('%Y%m%d')
        img.save(os.path.join(symbol_dir, f"{date_str}.png"))

    print(f"✅ Đã xử lý xong symbol: {symbol}")


In [ ]:
import pandas as pd
input_file = "SP500_values.csv"
df = pd.read_csv(input_file, usecols=["Symbol", "Date", "Close"])
df_goog = df[df["Symbol"] == "GOOG"][["Date", "Close"]]

output_file = "GOOG_filtered.csv"
df_goog.to_csv(output_file, index=False)

print(f"OK: {output_file}")


In [ ]:
import pandas as pd
import numpy as np
import os
from pyts.image import GramianAngularField
from sklearn.preprocessing import MinMaxScaler
from PIL import Image

seq_len = 96
input_file = 'dataset/sp500/GOOG_filtered.csv' 
output_dir = 'new_images_folder'
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(input_file, usecols=['date', 'Close'])
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Normalize
close_prices = df['Close'].values
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

# Initial GAF
gasf = GramianAngularField(image_size=seq_len, method='summation')
gadf = GramianAngularField(image_size=seq_len, method='difference')

for i in range(len(normalized) - seq_len + 1):
    window = normalized[i:i + seq_len].reshape(1, -1)

    gasf_img = gasf.transform(window)[0]
    gadf_img = gadf.transform(window)[0]
    zeros_img = np.zeros_like(gadf_img)
    stacked = np.stack([gasf_img, gadf_img, zeros_img], axis=-1)
    stacked_scaled = ((stacked - np.min(stacked)) / (np.max(stacked) - np.min(stacked)) * 255).astype(np.uint8)
    img = Image.fromarray(stacked_scaled, mode='RGB')
    date_str = df.iloc[i + seq_len - 1]['date'].strftime('%Y%m%d')
    img.save(os.path.join(output_dir, f"{date_str}.png"))

print("Helloo Done")
